In [9]:
train_path = "./new_train/new_train"
test_path = "./new_val_in/new_val_in"
submission_path = "./sample_submission.csv"
submission_dir = "./submissions"

In [ ]:
# Train Val Split
TRAIN_SIZE = 0.75
VAL_SIZE = 0.25

# training config
NUM_EPOCH = 5
BATCH_SIZE = 128
LEARNING_RATE = 1e-3

In [65]:
import os
import pickle
import numpy as np
from glob import glob
import pandas as pd
from tqdm import tqdm
from typing import Tuple
import matplotlib.pyplot as plt

import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# Data

In [66]:
class ArgoverseDataset(Dataset):
    def __init__(self, data_path: str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
        self.transform = transform

        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.pkl_list)

    def __getitem__(self, idx):

        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
        
        if self.transform:
            data = self.transform(data)

        return data

In [67]:
train = ArgoverseDataset(data_path=train_path)
test = ArgoverseDataset(data_path=test_path)

In [ ]:
train_size = int(TRAIN_SIZE * len(train))
test_size = len(full_dataset) - train_size
train, val = torch.utils.data.random_split(train, [train_size, test_size])

In [68]:
def my_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp, out = [], []
    for scene in batch:
        agent_idx = np.where(scene["agent_id"] == np.unique(scene["track_id"].flatten()))[0][0]
        inp.append(np.hstack([scene['p_in'][agent_idx], scene['v_in'][agent_idx]]))
        # out.append(np.hstack([scene['p_out'][agent_idx], scene['v_out'][agent_idx]]))
        out.append(scene['p_out'][agent_idx])

    inp = torch.FloatTensor(inp)
    out = torch.FloatTensor(out)
    return [inp, out]

train_loader = DataLoader(train, batch_size=BATCH_SIZE, shuffle = False, collate_fn=my_collate, num_workers=0)
val_loader = DataLoader(val, batch_size=BATCH_SIZE, shuffle = False, collate_fn=my_collate, num_workers=0)
test_loader = DataLoader(test, batch_size=BATCH_SIZE, shuffle = False, collate_fn=my_collate, num_workers=0)
exmaple = iter(train_loader)
exmaple.next()[0][0]

tensor([[3.2354e+03, 1.9295e+03, 2.5700e-01, 3.1313e-01],
        [3.2359e+03, 1.9298e+03, 4.6091e+00, 3.1098e+00],
        [3.2359e+03, 1.9299e+03, 8.8960e-01, 1.0443e+00],
        [3.2373e+03, 1.9315e+03, 1.3368e+01, 1.6047e+01],
        [3.2378e+03, 1.9317e+03, 5.1234e+00, 2.2956e+00],
        [3.2383e+03, 1.9323e+03, 4.6713e+00, 5.8263e+00],
        [3.2387e+03, 1.9326e+03, 4.5387e+00, 3.1254e+00],
        [3.2391e+03, 1.9327e+03, 3.4147e+00, 1.0973e+00],
        [3.2396e+03, 1.9335e+03, 5.7870e+00, 7.9927e+00],
        [3.2402e+03, 1.9340e+03, 5.1474e+00, 5.0185e+00],
        [3.2406e+03, 1.9344e+03, 4.0382e+00, 3.1020e+00],
        [3.2410e+03, 1.9350e+03, 4.8599e+00, 6.0268e+00],
        [3.2415e+03, 1.9354e+03, 5.0286e+00, 4.8091e+00],
        [3.2420e+03, 1.9357e+03, 4.7979e+00, 2.0820e+00],
        [3.2424e+03, 1.9359e+03, 3.4249e+00, 2.8949e+00],
        [3.2428e+03, 1.9363e+03, 4.7367e+00, 3.7335e+00],
        [3.2431e+03, 1.9365e+03, 2.8242e+00, 1.9251e+00],
        [3.243

In [18]:
submission = pd.read_csv(submission_path)
submission

,ID,v1,v2,v3,v4,v5,v6,v7,v8,v9,...,v51,v52,v53,v54,v55,v56,v57,v58,v59,v60
0,10002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10015,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10019,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10028,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3195,9897,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3196,99,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3197,9905,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3198,9910,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Training

In [75]:
# model training
is_cuda = torch.cuda.is_available()
if is_cuda:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
device

device(type='cuda')

In [76]:
class RMSELoss(torch.nn.Module):
    """RMSE Loss"""
    def __init__(self):
        super(RMSELoss, self).__init__()

    def forward(self, yhat, y):
        criterion = nn.MSELoss()
        loss = torch.sqrt(criterion(yhat, y))
        return loss

## Seq2Seq (LSTM)

In [77]:
# training config
ROLLOUT_LEN = 30

# model config
INPUT_SIZE = 4
EMBEDDING_SIZE = 8
HIDDEN_SIZE = 16
OUTPUT_SIZE = 2

In [78]:
class EncoderLSTM(nn.Module):
    def __init__(self, input_size, embedding_size=EMBEDDING_SIZE, hidden_size=HIDDEN_SIZE):
        super(EncoderLSTM, self).__init__()
        self.hidden_size = hidden_size

        self.linear = nn.Linear(input_size, embedding_size)
        self.lstm = nn.LSTMCell(embedding_size, hidden_size)

    def init_hidden(self, batch_size, hidden_size):
        # Initialize encoder hidden state
        return (
            torch.zeros(batch_size, hidden_size).to(device),
            torch.zeros(batch_size, hidden_size).to(device),
        )
    
    def forward(self, X):
        init_hidden = self.init_hidden(X.shape[0], self.hidden_size)
        
        embedded = F.relu(self.linear(X))
        hidden_state = self.lstm(embedded, init_hidden)
        return hidden_state

In [79]:
class DecoderLSTM(nn.Module):
    def __init__(self, embedding_size=EMBEDDING_SIZE, hidden_size=HIDDEN_SIZE, output_size=OUTPUT_SIZE):
        super(DecoderLSTM, self).__init__()
        self.hidden_size = hidden_size
        
        self.linear1 = nn.Linear(output_size, embedding_size)
        self.lstm1 = nn.LSTMCell(embedding_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, output_size)

    def forward(self, X, encoder_hidden):        
        embedded = F.relu(self.linear1(X))
        hidden = self.lstm1(embedded, encoder_hidden)
        output = self.linear2(hidden[0])
        return output, hidden

In [80]:
# model training
encoder = EncoderLSTM(INPUT_SIZE)
decoder = DecoderLSTM()
encoder.to(device)
decoder.to(device)

loss_fn = RMSELoss()
encoder_optimizer = torch.optim.Adam(encoder.parameters(), lr=LEARNING_RATE)
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=LEARNING_RATE)

In [ ]:
# training
for epoch in range(NUM_EPOCH):
    for i_batch, batch_data in enumerate(train_loader):
        inp, out = batch_data
        inp = inp.to(device)
        out = out.to(device)
        
        # Set to train mode
        encoder.train()
        decoder.train()

        # Initialize losses
        loss = 0

        # Encoder
        batch_size = inp.shape[0]
        input_length = inp.shape[1]
        output_length = out.shape[1]
        input_shape = inp.shape[2]

        # Get relative position
        initial_p_in = inp[:, 0, :2].detach().clone()
        inp[:, :, :2] = inp[:, :, :2] - initial_p_in[:, None]
        
        # Encode observed trajectory
        for i in range(input_length):
            encoder_input = inp[:, i, :]
            encoder_hidden = encoder(encoder_input)
        
        # Initialize decoder input with last coordinate in encoder
        decoder_hidden = encoder_hidden
        decoder_input = encoder_input[:, :2]
        decoder_outputs = torch.zeros(out.shape).to(device)

        # Decode hidden state in future trajectory
        for i in range(output_length):
            decoder_output, decoder_hidden = decoder(decoder_input, encoder_hidden)
            decoder_outputs[:, i, :] = decoder_output

            # Update loss
            loss += loss_fn(decoder_output[:, :2], out[:, i, :2] - initial_p_in)

            # Use own predictions as inputs at next step
            decoder_input = decoder_output

        # Get average loss for pred_len
        loss = loss / output_length

        # Backpropagate
        loss.backward()
        encoder_optimizer.step()
        decoder_optimizer.step()
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        if i_batch % 200 == 0:
            print(f"Epoch {epoch+1}/{NUM_EPOCH}, batch {i_batch}/{len(train_loader)}, loss: {loss.item():.4f}")

Epoch 1/5, batch 0/1609, loss: 23.1653
Epoch 1/5, batch 200/1609, loss: 19.4773
Epoch 1/5, batch 400/1609, loss: 16.6437
Epoch 1/5, batch 600/1609, loss: 16.3441
Epoch 1/5, batch 800/1609, loss: 14.7987
Epoch 1/5, batch 1000/1609, loss: 13.3119
Epoch 1/5, batch 1200/1609, loss: 12.4540
Epoch 1/5, batch 1400/1609, loss: 12.5594
Epoch 1/5, batch 1600/1609, loss: 12.3229
Epoch 2/5, batch 0/1609, loss: 11.7848
Epoch 2/5, batch 200/1609, loss: 11.0003
Epoch 2/5, batch 400/1609, loss: 9.4281
Epoch 2/5, batch 600/1609, loss: 9.9526
Epoch 2/5, batch 800/1609, loss: 9.3272
Epoch 2/5, batch 1000/1609, loss: 8.2274
Epoch 2/5, batch 1200/1609, loss: 7.6766
Epoch 2/5, batch 1400/1609, loss: 8.2957
Epoch 2/5, batch 1600/1609, loss: 8.0073
Epoch 3/5, batch 0/1609, loss: 7.9661
Epoch 3/5, batch 200/1609, loss: 7.4366
Epoch 3/5, batch 400/1609, loss: 6.4611
Epoch 3/5, batch 600/1609, loss: 7.1600
Epoch 3/5, batch 800/1609, loss: 6.8339
Epoch 3/5, batch 1000/1609, loss: 5.9722
Epoch 3/5, batch 1200/1609

In [55]:
initial_p_in[10]

tensor([3416.7327, 1842.6482])

In [ ]:
# predict 
with torch.no_grad():
    for i_batch, batch_data in enumerate(test_loader):
        inp, out = batch_data
        inp = inp.to(device)
        out = out.to(device)

        # Set to eval mode
        encoder.eval()
        decoder.eval()

        # Encoder
        batch_size = inp.shape[0]
        input_length = inp.shape[1]
        output_length = out.shape[1]
        input_shape = inp.shape[2]

        # Get relative position
        initial_p_in = inp[:, 0, :2].detach().clone()
        inp[:, :, :2] = inp[:, :, :2] - initial_p_in[:, None]

        # Encode observed trajectory
        for i in range(input_length):
            encoder_input = inp[:, i, :]
            encoder_hidden = encoder(encoder_input)

        # Initialize decoder input with last coordinate in encoder
        decoder_hidden = encoder_hidden
        decoder_input = encoder_input[:, :2]
        decoder_outputs = torch.zeros(out.shape)

        # Decode hidden state in future trajectory
        for i in range(output_length):
            decoder_output, decoder_hidden = decoder(decoder_input, encoder_hidden)
            decoder_outputs[:, i, :] = decoder_output + initial_p_in

            # Use own predictions as inputs at next step
            decoder_input = decoder_output


# Submission

In [ ]:
# make final submission dataframe


In [ ]:
from datetime import datetime
def save_submission(df, filename):
    filename = filename + "_" + str(datetime.now()) + ".csv"
    file_path = os.path.join(submission_dir, filename)
    df.to_csv(file_path, index=False)

In [ ]:
save_submission(df, "seq2seq(lstm)")